### Token Transfer Data Extraction

Data Extraction Notes => 

1. This workbook is used to extract the AAVE transfer data on ETH blockchain. 
2. The original data source is from a Kaggle live dataset called "ethereum-blockchain". 
3. To extract the dataset, we need to create a project on Google cloud and extract the data using bigquery. 
4. This workbook can be ran using Kaggle notebook. For the assessment purpose, I have saved each of the datasets to csv.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra (version: 1.23.2)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) (version: 1.5.0)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Token Transfer Data Querying

AAVE Transfer data querying (from Google Big Query) using Kaggle's public dataset BigQuery integration 

In [ ]:
import os
import time
import zipfile
from google.cloud import bigquery
import numpy as np # version: 1.23.2
import pandas as pd # version: 1.5.0
from tqdm import tqdm # version: 4.46.1
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project = "comp7860-project")

# Construct a reference to the "crypto_ethereum" dataset (https://www.kaggle.com/bigquery/ethereum-blockchain)
dataset_ref = client.dataset("crypto_ethereum", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "crypto_ethereum" dataset
tables = list(client.list_tables(dataset))

In [ ]:
def query_to_csv(sql, output_path): 
    df = client.query(sql).to_dataframe(progress_bar_type='tqdm_notebook')
    df.to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path), compression='gzip')

In [ ]:
sql = '''
SELECT token_address, from_address, to_address,block_timestamp, cast(value AS NUMERIC) FROM 
`bigquery-public-data.crypto_ethereum.token_transfers` 
WHERE token_address = "0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9"
'''
df = client.query(sql).to_dataframe(progress_bar_type='tqdm_notebook')

#### Data Cleaning

In [ ]:
# data cleaning
df.rename(columns={'f0_':'value'}, inplace = True)
df = df.dropna()
df['value'] = df['value'].apply(lambda x: float(x))
df['timestamp'] = pd.to_datetime(df['block_timestamp'])
df['timestamp'] = df['timestamp'].apply(lambda x: str(x)[:10])
df['timestamp'] = pd.to_datetime(df['timestamp'])
# new data period 
df_new = df[df['timestamp']>'2021-07-10']
df_new = df[df['timestamp']<'2022-07-09']
# existing data period  
df_existing = df[df['timestamp']>'2020-10-10']
df_existing = df[df['timestamp']<'2021-10-09']

In [ ]:
# read the new dataset into csv
df_new.to_csv("Aave_Raw_Transfer_Data_New.csv", index=False)
df_existing.to_csv('Aave_Raw_Transfer_Data.csv', index=False)

In [ ]:
df_new = df_new.drop(columns = ['token_address','block_timestamp'])
df_existing = df_existing.drop(columns = ['token_address','block_timestamp'])

In [ ]:
# add values between the 2 same addresses together
df_new[['from_address', 'to_address']] = np.sort(df_new[['from_address', 'to_address']], axis=1)
df_new = df_new.groupby(['timestamp','from_address','to_address']).agg(lambda x: sum(x)).reset_index()

df_existing[['from_address', 'to_address']] = np.sort(df_existing[['from_address', 'to_address']], axis=1)
df_existing = df_existing.groupby(['timestamp','from_address','to_address']).agg(lambda x: sum(x)).reset_index()

In [ ]:
# save processed datasets
df_new.to_csv('New_Data/AAVE_transaction_data_after_preprocessing_New.csv', index=False)
df_existing.to_csv('Existing_Data/AAVE_transaction_data_after_preprocessing.csv', index=False)